In [110]:
import numpy as np
from qutip import *
import matplotlib.pyplot as plt 
import oct2py
import math

oc = oct2py.Oct2Py()

In [111]:
# Initial states (Bell Diagonal states)
b1 = bell_state('00')
b2 = bell_state('01')
b3 = bell_state('10')
b4 = bell_state('11')

#States of bath qubits
e1 = tensor(maximally_mixed_dm(2), maximally_mixed_dm(2))
e2 = tensor(maximally_mixed_dm(2), maximally_mixed_dm(2), maximally_mixed_dm(2))

# Total initial state:
i1 = tensor(ket2dm(b1), e1, e2)

# Definitions
sx = sigmax()
sy = sigmay()
sz = sigmaz()

#Parameters for Hamiltonian (Couplngs etc)

#Coupling between two qubits
J_12 = 10

#Coupling between qubit and first layer of environment
J_23 = 20

#Coupling between first and second layer of environment 
J_34 = 30

#Chemical potential terms:
o_1 = 0
o_2 = 0
o_3 = 0
o_4 = 0


# Scrambling Time
#tau = 

# First environement evolution time
t = 1

In [132]:
# Hamiltonians

# System
H_12 = J_12/4 *tensor(sx, sx, qeye(2), qeye(2), qeye(2), qeye(2), qeye(2))

# Coupling between system and first environment
H_23 = J_23/4 *(tensor(qeye(2), sz, sz, qeye(2), qeye(2), qeye(2), qeye(2)) + tensor(qeye(2), sz, qeye(2), sz, qeye(2), qeye(2), qeye(2)))

# Coupling between two layers of environment
H_34 = J_34/4 *(tensor(qeye(2), qeye(2), sz, qeye(2), sz, qeye(2), qeye(2)) +
                 tensor(qeye(2), qeye(2), sz, qeye(2), qeye(2), sz, qeye(2)) +
                 tensor(qeye(2), qeye(2), sz, qeye(2), qeye(2), qeye(2), sz) +
                 tensor(qeye(2), qeye(2), qeye(2), sz, sz, qeye(2), qeye(2)) +
                 tensor(qeye(2), qeye(2), qeye(2), sz, qeye(2), sz, qeye(2)) +
                 tensor(qeye(2), qeye(2), qeye(2), sz, qeye(2), qeye(2), sz))

In [133]:
# Initial psuedopure state (Wigner state)

zhi=8

sys_init = (0.25*(1-(zhi/8))*tensor(qeye(2), qeye(2))) + (zhi/8 * ket2dm(b3))

discord_states = [sys_init.full()]
otoc_states = [sys_init]

In [134]:

tau_list=[0]

for tau in np.linspace(0.01, 1, 100):
    # Full initial state
    full_init = tensor(sys_init, maximally_mixed_dm(2), maximally_mixed_dm(2), maximally_mixed_dm(2), maximally_mixed_dm(2), maximally_mixed_dm(2))

    # Time evolution

    # Evolution under coupling with first environment
    r1= mesolve(H_12+H_23, full_init, np.linspace(0, t, 500))
    # Saving final state to use as initial state in next step
    r2i = r1.states[len(r1.states)-1]

    # Scrambling (half)
    r2= mesolve(H_12+H_23+H_34, r2i, np.linspace(0, tau/2, 500))
    r3i = r2.states[len(r2.states)-1]

    # Pi pulse on system 
    r3 = mesolve(0.5*(tensor(sx, qeye(2), qeye(2), qeye(2), qeye(2), qeye(2), qeye(2)) + tensor(qeye(2), sx, qeye(2), qeye(2), qeye(2), qeye(2), qeye(2))), r3i, np.linspace(0, np.pi, 500))
    r4i = r3.states[len(r3.states)-1]

    # Scrambling (second half)
    r4 = mesolve(H_12+H_23+H_34, r4i, np.linspace(0, tau/2, 500))
    r5i = r4.states[len(r4.states)-1]
    # Saving reduced state for calculating discord
    discord_states.append(r5i.ptrace([0,1]).full())

    # First environment coupling evolution
    r5 = mesolve(H_12+H_23, r5i, np.linspace(0, t, 500))
    r6i = r5.states[len(r5.states)-1]
    # Saving reduced state for calculating OTOC
    otoc_states.append(r6i.ptrace([0,1]))
    tau_list.append(tau)
    print(tau)



0.01
0.02
0.03
0.04
0.05
0.060000000000000005
0.06999999999999999
0.08
0.09
0.09999999999999999
0.11
0.12
0.13
0.14
0.15000000000000002
0.16
0.17
0.18000000000000002
0.19
0.2
0.21000000000000002
0.22
0.23
0.24000000000000002
0.25
0.26
0.27
0.28
0.29000000000000004
0.3
0.31
0.32
0.33
0.34
0.35000000000000003
0.36000000000000004
0.37
0.38
0.39
0.4
0.41000000000000003
0.42000000000000004
0.43
0.44
0.45
0.46
0.47000000000000003
0.48000000000000004
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.5700000000000001
0.5800000000000001
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.6900000000000001
0.7000000000000001
0.7100000000000001
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.8200000000000001
0.8300000000000001
0.8400000000000001
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.9400000000000001
0.9500000000000001
0.9600000000000001
0.97
0.98
0.99
1.0


In [135]:
print(len(otoc_states), len(tau_list))

101 101


In [136]:
otoc=[]
discord = []

# Calculating discord and OTOC
for i in range(0, len(otoc_states)):
#     OTOC is calculated manually from the 
    num = (otoc_states[i]*otoc_states[0]).tr()
    den = ((otoc_states[i]*otoc_states[i]).tr()*(otoc_states[0]*otoc_states[0]).tr())**(0.5)
    otoc_calc = num/den
    otoc.append(otoc_calc)
    
    disc_calc = oc.discord(discord_states[i], 200, 200)
    discord.append(disc_calc)
    

In [137]:
# plotting discord and otoc
plt.plot(tau_list, otoc, linestyle='dashed', markersize='0.5')
plt.xlabel('Scrambling Time')
plt.ylabel('OTOC')
plt.savefig('scrambling OTOC sx-sz', format='eps', dpi=1000)
plt.clf()

plt.plot(tau_list, discord, linestyle='dashed', markersize='0.5')
plt.xlabel('Scrambling Time')
plt.ylabel('Discord')
plt.savefig('scrambling discord sx-sz', format='eps', dpi=1000)
plt.clf()

<Figure size 432x288 with 0 Axes>